### Workflow for simulating a protein-ligand complex in water using OpenMM with Open Force Field/AMBER
Openmm is a toolkit for high performance molecular simulations. OpenMM is not an application in the traditional sense, with simulations being run using python scripts, done via utilising the application layer - which is just a set of Python libraries.

This tutorial demonstrates how to create and run a simple MD simulation of a protein-ligand complex in water, specifically, a potential drug molecule in the binding site of COVID-19's main protease. (The chosen drug molecule is arbitrary, chosen from a set of linked fragment screens which were generated with DeLinker.)[1]

In order to do this, both the protein and ligand must be parameterised, protonated and combined.

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import MolDrawing, DrawingOptions
from rdkit.Chem import MolStandardize

In [3]:
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField as OFFForceField
from openmmforcefields.generators import SystemGenerator
from simtk import openmm, unit
from simtk.openmm import app
from simtk.openmm.app import PDBFile
from simtk.openmm.app.modeller import Modeller
from simtk.openmm.app import NoCutoff, HBonds

/home/cree/anaconda3/envs/openff/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Importing this package as `import openforcefield.XXX` and `from openforcefield import XXX` was marked for deprecation in version `0.8.3`. From version `0.9.0` onwards this package will need to be imported as `import openff.toolkit.XXX` and `from openff.toolkit import XXX`. See the `0.8.3` release notes for more information.
  """Entry point for launching an IPython kernel.


In [4]:
from pdbfixer import PDBFixer
import pdb4amber
import parmed

### Ligand preparation
The ligand was generated using DeLinker without hydrogens, so the first step is to add them. 

Once hydrogens are added, the ligand can be parameterised with OpenForceField. 

Once it's parameterised, a ParmEd structure object can be created which allows the ligand to be combined with a similar protein structure object, which can then be used to create a system for an openmm simulation.

------------------------------------------------------------------------------------------------------------------

First, hydrogens are added using obabel. (! at the beginning of a cell runs the command in bash.) 

Obabel is a tool that allows interconversion between different file formats.

In [5]:
#adding hydrogens to ligand (ligand.pdb is delinker output)
!obabel ligand.pdb -ipdb -opdb -O ligand_h.pdb -h

1 molecule converted


.pdb files lack bond connectivity information, which is needed to parameterise the ligand. The .sdf file format has the information required. 

The .pdb file can be readily converted into .sdf using obabel.

In [6]:
#convert to sdf
!obabel ligand_h.pdb -ipdb -osdf -O ligand_h.sdf

1 molecule converted


Now the protonated ligand files can both be loaded into openmm.

(Warnings are just because each atom does not have a unique name and can be ignored.)

In [7]:
#create pdb object
pdbfile = PDBFile('./ligand_h.pdb')
#create molecule object
mol = Molecule.from_file('./ligand_h.sdf')

/home/cree/anaconda3/envs/openff/lib/python3.7/site-packages/simtk/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (HETATM   35  H   UNK     0      15.798  -5.475  19.565  1.00  0.00           H  , HETATM   34  H   UNK     0      16.517  -6.551  20.738  1.00  0.00           H  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/cree/anaconda3/envs/openff/lib/python3.7/site-packages/simtk/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (HETATM   36  H   UNK     0      14.971  -6.944  20.025  1.00  0.00           H  , HETATM   35  H   UNK     0      15.798  -5.475  19.565  1.00  0.00           H  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/cree/anaconda3/envs/openff/lib/python3.7/site-packages/simtk/openmm/app/internal/pdbstruct

There are three main components to OpenMM: System, Simulation, and Context.

System is a collection of interacting particles and it defines the mass of each particle, specifies distance constraints and contains a list of force objects that define the interactions. This is the first class to be created. (A force object contains anything that affects the systems behaviour e.g. forces, thermostats, barostats, etc.)

The ligand can now be parameterised by creating a system object using openff.

In [8]:
# Create the Open Force Field Topology from an OpenMM Topology object.
omm_topology = pdbfile.topology
off_topology = Topology.from_openmm(omm_topology, unique_molecules=[mol])

# Load the OpenFF "Parsley" force field.
forcefield = OFFForceField('openff-1.0.0.offxml')

# Parametrize the topology and create an OpenMM System.
system = forcefield.create_openmm_system(off_topology)

A structure object can now be created, which is used to combine the ligand with other structures, in this case, the protein.

The Structure class utilised by the ParmEd package makes it easy to quickly and safely manipulate a chemical system, its underlying topology, and force field parameters describing its potential energy function. 

It is used in this workflow to combine the protein (parameterised with AMBER) and ligand (paramterised with openff) into one system that can be simulated with openmm.

In [9]:
omm_topology = pdbfile.getTopology()
positions = pdbfile.getPositions()
# Convert OpenMM System into a ParmEd Structure.
ligand_structure = parmed.openmm.load_topology(omm_topology,
                                                system,
                                                xyz=positions)

### Protein preparation


Before the protein can be used in openmm it must first be checked, and if necessary, fixed. Two tools will be used for this step.

First, a tool called PDBFixer will find missing residues, rename nonstandard residues etc, and then pdb4amber will then choose alternate residues and add hydrogens (and making sure protonation occurs in the right place).

A summary of the changes made by pdb4amber to the protein can be found in 'pdb4amber.log'.

--------------------------------------------------------------------------------------------------------------------------

In [10]:
#fix PDB
fixer = PDBFixer('./mpro.pdb')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
PDBFile.writeFile(fixer.topology, fixer.positions, open('mpro_nosolvent.pdb', 'w'))

In [11]:
#more PDB fixing + protonation
pdb4amber.run(arg_pdbout='mproh.pdb', arg_pdbin='mpro_nosolvent.pdb', arg_reduce=True, arg_logfile='pdb4amber.log')
!tail -n 20 pdb4amber.log

To generate the protein system, a system generator must be used, along with some forcefield constraints.

A system generator facilitates getting parameters on a ligand using openff whilst using an AMBER forcefield for the protein, allowing the combining of the ParmEd structures.

In [14]:
#make the system generator, load forcefields
forcefield_kwargs = { 'constraints' : app.HBonds, 'rigidWater' : True, 'removeCMMotion' : False, 'hydrogenMass' : 4*unit.amu }

#system generator used to create system objects
system_gen = SystemGenerator(forcefields=['amber99sbildn.xml', 'tip3pfb.xml'], 
                             small_molecule_forcefield="openff-1.3.0.offxml", 
                             forcefield_kwargs=forcefield_kwargs,
                            molecules=[mol,])

The next task is to solvate the protein using OpenMM's modeller class, which has various tools for editing molecular models.

First a modeller object must be created.

In [12]:
#make the modeller
pdbfile = PDBFile('mproh.pdb')
top = pdbfile.getTopology()
pos = pdbfile.getPositions()
modeller = app.Modeller(top, pos)

The protein can be solvated by calling the .addSolvent function.

In [15]:
#now add water
modeller.addSolvent(system_gen.forcefield, model="tip3p", padding=1, ionicStrength=0*unit.molar, neutralize=False)
top = modeller.topology
pos = modeller.positions

#write a pdb file of solvated protein
with open("mpro_h_solvent.pdb", 'w') as outfile:
    app.PDBFile.writeFile(modeller.topology, modeller.positions, outfile)

The fixed, solvated and protonated protein must be parameterised (using AMBER) before it can be used in OpenMM.

For parameterisation, the 'amber99sbildn' forcefield will be used.

In [16]:
#parameterize the protein and create a system 
system = system_gen.create_system(topology=modeller.topology)

The parameterised protein system can now be used to create a ParmEd structure object.

In [17]:
#convert the protein system into a ParmEd structure
protein_structure = parmed.openmm.load_topology(top,
                                           system,
                                           xyz=pos)

In [18]:
#write structure to pdb file
PDBFile.writeFile(protein_structure.topology, 
                  protein_structure.positions, 
                  open('protein_structure.pdb', 'w'))
pdbfile = PDBFile('protein_structure1.pdb')

### Combining the protein and ligand

(If you already have a protein-ligand structure from another notebook, you can load it using magic commands.) e.g. %store -r complex_structure

Now both the protein and ligand structures can be combined into a single structure, which can be used to create an openmm simulation of the protein-ligand complex (in water).

In [19]:
#adding the structures from protein.ipynb and ligand.ipynb
complex_structure = protein_structure + ligand_structure
#writing new structure to a .pdb file
complex_structure.write_pdb('./complex_system.pdb')

### Simulating the protein-ligand complex

Now an openmm system can be generated (from the previously defined system generator) from the protein-ligand structure.

In [21]:
#convert the structure to an openmm system
complex_system = system_gen.create_system(topology=complex_structure.topology)

A simulation, which is a combination of a system, integrator (how the equations of motion are advanced) and topology (atom coordinates) can now be created using the system that was just made.

The simulation creates a context, which stores the complete state of a simulation and contains information such as the positions and velocities of particles. Initially positions are randomised.

Parameters such as the temperature of the simulation can be specified.

In [24]:
#propagate the System with Langevin dynamics.
time_step = 1*unit.femtoseconds  # simulation timestep
temperature = 300*unit.kelvin  # simulation temperature
friction = 1/unit.picosecond  # collision rate
integrator_min = openmm.LangevinIntegrator(temperature, friction, time_step)

#set up an openmm simulation
simulation = openmm.app.Simulation(complex_structure.topology, complex_system, integrator_min)

#set the initial positions
positions = complex_structure.positions
simulation.context.setPositions(positions)

If the simulation was to be run as is, it would blow up due to extreme forces on inappropriately placed atoms. First the system must be minimised.

In [25]:
simulation.minimizeEnergy()
simulation.saveState('./minimized.state')

KeyboardInterrupt: 

This minimised system can be written to a .pdb file (and can be used as a starting point for multiple runs).

In [27]:
#get state of minimised simulation
state = simulation.context.getState(getPositions=True)
#get positions of minimised simulation
positions = state.getPositions()
#write minimised structure to .pdb
PDBFile.writeFile(complex_structure.topology, positions, open("minimised_complex.pdb", "w"))
pdbfile_min = PDBFile('./mminimised_complex.pdb')

/home/cree/anaconda3/envs/openff/lib/python3.7/site-packages/simtk/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (HETATM55751  H   UNK C   1      16.429  -3.892  20.548  1.00  0.00           H  , HETATM55750  H   UNK C   1      16.747  -5.242  21.647  1.00  0.00           H  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/cree/anaconda3/envs/openff/lib/python3.7/site-packages/simtk/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (HETATM55752  H   UNK C   1      15.326  -5.295  20.585  1.00  0.00           H  , HETATM55751  H   UNK C   1      16.429  -3.892  20.548  1.00  0.00           H  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/cree/anaconda3/envs/openff/lib/python3.7/site-packages/simtk/openmm/app/internal/pdbstruct

A new minimised structure object can be created and used as the starting point for our simulation.

In [28]:
#load minimised_complex pdb as structure
minimised_structure = parmed.openmm.load_topology(complex_structure.topology, complex_system,xyz=positions)

After the simulation has been minimized, a 'production run' can be performed. How often the output will be written can be set by variables and appending reporter objects to the simulation.

The results of the simulation will be written to trajectory_prod.pdb, which can be loaded into visualisation software e.g. VMD.

In [30]:
#propagate the System with Langevin dynamics.
time_step = 1*unit.femtoseconds  # simulation timestep
temperature = 300*unit.kelvin  # simulation temperature
friction = 1/unit.picosecond  # collision rate
integrator_prod = openmm.LangevinIntegrator(temperature, friction, time_step)

#length of the simulation.
num_steps = 1000  # number of integration steps to run

# Logging options.
trj_freq = 1  # number of steps per written trajectory frame
data_freq = 1  # number of steps per written simulation statistics

#set up an OpenMM simulation using minimised structure
simulation = openmm.app.Simulation(minimised_structure.topology, complex_system, integrator_prod)

#set the initial positions.
positions = pdbfile.getPositions() 
simulation.context.setPositions(positions)

#randomize the velocities from a Boltzmann distribution at a given temperature.
simulation.context.setVelocitiesToTemperature(temperature)

#configure the information in the output files.
pdb_reporter = openmm.app.PDBReporter('trajectory_prod.pdb', trj_freq)
state_data_reporter = openmm.app.StateDataReporter('data_prod.csv', data_freq, step=True,
                                                   potentialEnergy=True, temperature=True,
                                                   density=True)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)

Finally, run the simulation.

In [ ]:
import time

print("Starting simulation")
start = time.process_time()

#run the simulation
simulation.step(num_steps)

end = time.process_time()
print("Elapsed time %.2f seconds" % (end-start))
print("Done!")

[1] - https://pubs.acs.org/doi/10.1021/acs.jcim.9b01120